# Introduction to Bayesian Networks (BNs)



## CSCI E-83
## Stephen Elston

**Bayesian Networks** or **BNs** are one of the basic representations of probabilistic graphical models. BNs are **directed acyclic graphs** or **DAGs** which attempt to capture the **independence structure** of the distribution. Taking advantage of the independency structure of the distribution can greatly reduce the computational complexity of inference on the graphic model. 

The directed nature of the BN can capture **causality** in some cases. However, keep in mind, information on independence can flow both directions along the graph.



## A simple example

We have already worked on the student GRE score and letter problem. As a reminder, a student would like to make inferences or **queries** on the joint distribution $P(D,I,S,G,L)$. She can make the following assertions about the independencies for this problem: 

1. The degree of difficulty, D, of the machine learning course is **unconditionally independent** of all other variable, $\{D \bot I,S,G.L \}$.
2. The intelligence of the student is, I, **unconditionally independent** of all other variable, $\{I \bot D,S,G.L \}$.
3. The quality of the student's recommendation letter, L, is **conditionally independent**  of intelligence, her GRE score, and her letter, given her grade, $\{L \bot I,S,L\ |\ G\}$.
4. The student's grade in the machine learning course, G, is **conditionally independent** of her GRE score and her letter, give the difficulty of the course and her intelligence, $\{G  \bot S,L\ |\ I,D\}$. 
5. The students GRE scores, S, are **conditionally independent** of her grade, difficulty of the course and letter, given her intelligence, $\{S \bot G,D,L\ |\ I\}$.

It is readily possible to construct a BN that represents the dependencies on this distribution as shown in the figure below.  

<img src="img/LetterDAG.JPG" alt="Drawing" style="width:600px; height:300px"/>
<center> **DAG for the student score and letter distribution** </center>


In [7]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

In [8]:
## Define the network structure.
## The first value of the tuple defines the origin
## of the connector and the second the terminal point
## of the directed edge. 
student_model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

In [9]:
## Define the independent variables
CPD_D = TabularCPD(variable='D', variable_card=2, values=[[0.7, 0.3]])
print(CPD_D)
CPD_I = TabularCPD(variable='I', variable_card=2, values=[[0.8, 0.2]])
print(CPD_I)

╒═════╤═════╕
│ D_0 │ 0.7 │
├─────┼─────┤
│ D_1 │ 0.3 │
╘═════╧═════╛
╒═════╤═════╕
│ I_0 │ 0.8 │
├─────┼─────┤
│ I_1 │ 0.2 │
╘═════╧═════╛


In [10]:
# Define the distributions with a single conditional variable 
# or evidence. 
CPD_L = TabularCPD(variable='L', variable_card=2, 
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'], # Leter depends on the grade
                   evidence_card=[3])
print(CPD_L)

CPD_S = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'], # GRE score depneds on intelligence
                   evidence_card=[2])
print(CPD_S)

╒═════╤═════╤═════╤══════╕
│ G   │ G_0 │ G_1 │ G_2  │
├─────┼─────┼─────┼──────┤
│ L_0 │ 0.1 │ 0.4 │ 0.99 │
├─────┼─────┼─────┼──────┤
│ L_1 │ 0.9 │ 0.6 │ 0.01 │
╘═════╧═════╧═════╧══════╛
╒═════╤══════╤═════╕
│ I   │ I_0  │ I_1 │
├─────┼──────┼─────┤
│ S_0 │ 0.95 │ 0.2 │
├─────┼──────┼─────┤
│ S_1 │ 0.05 │ 0.8 │
╘═════╧══════╧═════╛


In [11]:
# Define the distributions with a multiple conditional variables 
# or evidence. 
CPD_G = TabularCPD(variable='G', variable_card=3, 
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])
print(CPD_G)

╒═════╤═════╤══════╤══════╤═════╕
│ I   │ I_0 │ I_0  │ I_1  │ I_1 │
├─────┼─────┼──────┼──────┼─────┤
│ D   │ D_0 │ D_1  │ D_0  │ D_1 │
├─────┼─────┼──────┼──────┼─────┤
│ G_0 │ 0.3 │ 0.05 │ 0.9  │ 0.5 │
├─────┼─────┼──────┼──────┼─────┤
│ G_1 │ 0.4 │ 0.25 │ 0.08 │ 0.3 │
├─────┼─────┼──────┼──────┼─────┤
│ G_2 │ 0.3 │ 0.7  │ 0.02 │ 0.2 │
╘═════╧═════╧══════╧══════╧═════╛


## Independency and uniqueness of BNs

We have just defined a BN with the Independency structure of the distribution defined by the assertions. The independency structure is quite important since by imposing independencies can greatly reduce the computational complexity of the graph.   

However, there is no guarantee that a given BN uniquely defines the independency structure of a distribution. A simple example of four BNs representing the same independency structure is shown in the figure below. In each case, the following **conditional independence** assertions are true:

$$A\ \bot\ C\ |\ B\\
Or,\\
C\ \bot\ A\ |\ B$$

<img src="img/Dependency.JPG" alt="Drawing" style="width:600px; height:300px"/>
<center> **Multiple BNs with same dependence structure** </center>

Each of these cases supports that   

1. **Causal or Cascade:** In this case A causes B which causes C. This information is represented in the directed edges. If B is an evidence variable then A and C are separated and cannot dependent. Once a value is assigned to B, A depends on B and C depends on B, but A and C are **decoupled**. 
2. **Evidential:** Once again, the evidence variable B separates A and C, just as they do in the causal case. In this case, evidence is added to the DAG which allows inference of A.   
3. **Common Evidence, V-Structure, or Collider:** In this case, having evidence on B **blocks** the path or **separates** A from C. In other words, knowing B explains away A and C. 
4. **Common Cause or Common Parent:** In this case, B is causal to both A and C, making A and C independent. 

A key point here is that any of these DAGs has the same aforementioned independence properties. In other words, multiple DAGs can exhibit the same independencies. 

We can write a generalization of the independence properties of a BN:

> **Definition:** On a graph $G$, the variable $X_i$ is independent of its **nondecendents** given its **parents**, $Pa_{X_i}$. We can say that $G$ includes as set of **local conditional independence assumptions**:

$$I_{\ell}(G): \{X_i\ \bot\ Nondecendants_{X_i}\ |\ Pa_{X_i}:\ \forall i\ \}$$

Keeping the above in mind, we can find the independencies of the DAG representing the distribution for the student letter and GRE score problem:

In [12]:
## Find the local independencies in the BN
# Getting all the local independencies in the network.
student_model.local_independencies(['D', 'I', 'S', 'G', 'L'])

(D _|_ I, L, S, G)
(I _|_ D, L, S, G)
(S _|_ D, L, G | I)
(G _|_ L, S | D, I)
(L _|_ D, I, S | G)

## DAGs and independence maps

We can generalize beyond the above example we can say that a DAG represents a probability distribution **consistent** with the DAG. We can factor the the graph:
$$P(X) = \prod_{i=1:d}P(x_i|X_{\pi_i})$$

Where, $P(X)$ is probability distribution, $X_{\pi_i}$ is the set of **parents** of $x_i$, a node on the graph, and $d$. In other words, in a DAG a node $x_i$ is independent of all other nodes give the set of the node's parents, $X_{\pi_i}$.

We can also generalize the idea of independencies of a DAG G by writing:

$$I(G) = \{ X \bot Z\ |\ Y:\ dsep_G(X:Z|Y) \} $$ 

This leads us to an important definition of the properties of DAGs:

> **Definition:** A DAG, G, is an **independence map** or **I-map** of a distribution P if $I_l(G) \subseteq I(P)$, where $I(P)$ is the set of independencies of the distribution P and $I_l(G)$ is the set of independencies of the DAG. 

We can see that there are multiple DAGs for which $I_l(G) \subseteq I(P)$ can be true.  But, what are the practical implications of this definition? 

From the above definition, you can see that it can be the case that a DAG can be an I-map, but not a complete representation of the independencies of $P$. This leads us to another definition:

>**Definition:** A DAG, G, is a **minimal I-map** for a distribution P if removal of even a single edge renders $G$ not an I-map. 

Note that a minimal I-map may not be unique for a distribution $P$. 

## Trails in DAGs

